# ECEN 4322-5322 Data and Network Science

## Title- Fare prediction for flights


#### Group Members - Chirag Chandrashekar, Chris Alexander, Viveka Salinamakki

## Introduction 

The dataset chosen for analysis is the itineraries of flights in the USA over 6 months. For exploratory data analysis, we aim to find the airport with the highest traffic or the best-connected airports, the price of flights during the different hours of the day, the average/minimum travel distance for which people prefer flight, whether an average flight is fully booked/percentage of flights booked, and popular airlines. The end goal here is to predict the fare of a flight. Due to the large number of rows and features, the prediction can provide a good estimation of the fare. Techniques such as data grouping and manipulation, visualization, regular expressions, data modeling, feature engineering, model validation, and prediction will help achieve our goal. 


<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Importing the Data

In [118]:
import numpy as np

import pandas as pd
from pandas.api.types import CategoricalDtype

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import zipfile
import os

from ecen5322_utils import run_linear_regression_test

# Plot settings
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12

### Sampling the dataset

Write how the data is read as chunks and sampled

In [119]:
'''
dfs=[]
with pd.read_csv("car_price_prediction.csv", chunksize=1000000) as reader: #read as chunks due to low system resorces to read 30gb file
    reader
    for chunk in reader:
        #print(type(chunk))
        data_index=chunk.index
        shuffled_indices = np.random.permutation(data_index)  #shuffling and sampling data to reduce the data to 500000 entries
        #print(chunk.loc[shuffled_indices])
        #print(type(chunk))
        chunk,leftover= np.split(chunk.loc[shuffled_indices],[12000]) #selects first N rows from each chunk
        #print(type(chunk))
        #print(chunk)
        #chunks=chunk.to_frame
        #print(chunks)
        dfs.append(chunk) #makes a list of dataframe chunks
        #joined_chunk=pd.concat(chunk)
        #print(joined_chunk)
final_df=pd.concat(dfs) #concats all the chunks in the list of dataframe
print("final dataframe")
final_df
'''

'\ndfs=[]\nwith pd.read_csv("car_price_prediction.csv", chunksize=1000000) as reader: #read as chunks due to low system resorces to read 30gb file\n    reader\n    for chunk in reader:\n        #print(type(chunk))\n        data_index=chunk.index\n        shuffled_indices = np.random.permutation(data_index)  #shuffling and sampling data to reduce the data to 500000 entries\n        #print(chunk.loc[shuffled_indices])\n        #print(type(chunk))\n        chunk,leftover= np.split(chunk.loc[shuffled_indices],[12000]) #selects first N rows from each chunk\n        #print(type(chunk))\n        #print(chunk)\n        #chunks=chunk.to_frame\n        #print(chunks)\n        dfs.append(chunk) #makes a list of dataframe chunks\n        #joined_chunk=pd.concat(chunk)\n        #print(joined_chunk)\nfinal_df=pd.concat(dfs) #concats all the chunks in the list of dataframe\nprint("final dataframe")\nfinal_df\n'

In [120]:
#Load data to new csv file
#final_df
'''
data_index2=final_df.index
shuffled_indices2 = np.random.permutation(data_index2)
final_df2,leftover2=np.split(final_df.loc[shuffled_indices2],[500000])
final_df2
'''

'\ndata_index2=final_df.index\nshuffled_indices2 = np.random.permutation(data_index2)\nfinal_df2,leftover2=np.split(final_df.loc[shuffled_indices2],[500000])\nfinal_df2\n'

In [121]:
#from google.colab import  files
#final_df2.to_csv('sampled_file.csv')
#files.download('sampled_file.csv')

In [122]:
#data=pd.read_csv('sampled_file.csv')

#New read_csv function to read empty values as -1 and remove unamed column
data=pd.read_csv("sampled_file.csv"
                 #, dtype=str
                 , keep_default_na=True
                 , na_values=-1
                 , na_filter=True).drop(columns=['Unnamed: 0'])
data

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,e95cef0009893d65558d17324e468aea,2022-06-01,2022-06-15,PHL,JFK,SUAJZNB3,PT5H17M,0,True,False,...,1655309340||1655323200,2022-06-15T12:09:00.000-04:00||2022-06-15T16:00:00.000-04:00,BOS||JFK,PHL||BOS,American Airlines||American Airlines,AA||AA,Airbus A321||AIRBUS INDUSTRIE A321 SHARKLETS,5160||4920,280||185,coach||coach
1,778d47d0785023302cd075d735d27db8,2022-08-11,2022-08-20,CLT,ATL,TA7NA0MC,PT14H,1,False,False,...,1661037300||1661050560||1661081220,2022-08-20T19:15:00.000-04:00||2022-08-20T22:56:00.000-04:00||2022-08-21T07:27:00.000-04:00,DTW||IND||ATL,CLT||DTW||IND,Delta||Delta||Delta,DL||DL||DL,Boeing 717||Airbus A321||Boeing 737-900,6480||4020||5220,505||241||434,coach||coach||coach
2,55ca7cc1d822f310963a25d84656bb47,2022-06-26,2022-07-01,OAK,JFK,HA0NA0MC,PT8H13M,0,False,False,...,1656706200||1656730740,2022-07-01T13:10:00.000-07:00||2022-07-01T22:59:00.000-04:00,LAX||JFK,OAK||LAX,Delta||Delta,DL||DL,Embraer 175 (Enhanced Winglets)||,5040||20040,338||2458,coach||coach
3,598db4391eb0bc5752b09c871111a5ce,2022-05-14,2022-05-23,BOS,EWR,QAA3OFEN,PT1H41M,0,False,False,...,1653347940,2022-05-23T19:19:00.000-04:00,EWR,BOS,United,UA,Airbus A319,6060,185,coach
4,e09441b23c76e8a35be3acb61d8e3e95,2022-09-25,2022-10-13,ATL,BOS,TAUNX0BC,PT10H32M,0,True,False,...,1665667500||1665697620,2022-10-13T09:25:00.000-04:00||2022-10-13T17:47:00.000-04:00,JFK||BOS,ATL||JFK,Delta||Delta,DL||DL,Boeing 737-900||Airbus A220-100,7800||5220,762||185,coach||coach
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,cfd0ca3c7574a87f1ce1dfa83d1a053b,2022-07-02,2022-07-15,LAX,CLT,HA7OA0MQ,PT9H10M,1,False,False,...,1657969320||1657986300,2022-07-16T07:02:00.000-04:00||2022-07-16T11:45:00.000-04:00,DTW||CLT,LAX||DTW,Delta||Delta,DL||DL,Airbus A321||Boeing 717,16020||6300,1985||505,coach||coach
499996,3c671c288391b26e5888c84ef143424d,2022-05-04,2022-06-23,CLT,LAX,UAVNA0MC,PT14H11M,0,False,False,...,1655982720||1655995500||1656029760,2022-06-23T07:12:00.000-04:00||2022-06-23T09:45:00.000-05:00||2022-06-23T17:16:00.000-07:00,ATL||DFW||LAX,CLT||ATL||DFW,Delta||Delta||Delta,DL||DL||DL,Boeing 717||Airbus A321||Airbus A320,4020||8100||11760,228||725||1238,coach||coach||coach
499997,2313cf9772214eaddf14651c8847eaeb,2022-08-14,2022-08-26,BOS,OAK,QAA0OHEN,PT11H11M,0,False,False,...,1661558040||1661573100||1661583120,2022-08-26T18:54:00.000-05:00||2022-08-26T22:05:00.000-06:00||2022-08-26T23:52:00.000-07:00,IAH||SLC||OAK,BOS||IAH||SLC,United||Delta||Delta,UA||DL||DL,Airbus A320||Airbus A319||Airbus A220-100,15180||11460||6780,1602||1204||588,coach||coach||coach
499998,9141b51058bfbdf9adc91abb2be9078c,2022-08-09,2022-09-06,EWR,SFO,L7AHZNN1,PT8H13M,0,False,False,...,1662485760||1662506040,2022-09-06T12:36:00.000-05:00||2022-09-06T16:14:00.000-07:00,ORD||SFO,EWR||ORD,American Airlines||American Airlines,AA||AA,Boeing 737-800||Boeing 737-800,9300||16740,720||1847,coach||coach


In [123]:
#analysis of Chris's columns
Chris_columns=data.iloc[:,9:19]
print(Chris_columns)
specific_col= Chris_columns.iloc[:,9]
specific_col.value_counts()
#.value_counts().sort_values()
#.sort_values()
#analysis of Chirag's columns
Chirag_columns=data.iloc[:,0:9]
print(Chirag_columns)
#specific_col1= Chirag_columns.iloc[:,0:1]
#print(specific_col1.value_counts())
print(data.iloc[:,0:28].isna().sum())



#Viveka columns analysis
#data.columns
print(data.iloc[:,-8:].isna().sum())
data.iloc[:,-8:]
pattern=r"([^||]+)"
#data['segmentsAirlineCode'].str.extractall()

        isRefundable  isNonStop  baseFare  totalFare  seatsRemaining  \
0              False      False    160.93     196.60               7   
1              False      False    294.89     349.21               3   
2              False      False    430.70     486.60               1   
3              False       True    189.77     218.60               9   
4              False      False    212.09     257.20               9   
...              ...        ...       ...        ...             ...   
499995         False      False    488.37     548.60               7   
499996         False      False    373.95     434.20               7   
499997         False      False    579.53     651.10               9   
499998         False      False    310.70     357.60               3   
499999         False      False    659.54     737.11               3   

        totalTravelDistance   segmentsDepartureTimeEpochSeconds  \
0                     465.0              1655304180||1655318280   
1

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Cleaning the data

### make NaN =0  

In [124]:
data = data.dropna(subset=['totalTravelDistance'])
data
print(data.iloc[:,0:28].isna().sum())

legId                                   0
searchDate                              0
flightDate                              0
startingAirport                         0
destinationAirport                      0
fareBasisCode                           0
travelDuration                          0
elapsedDays                             0
isBasicEconomy                          0
isRefundable                            0
isNonStop                               0
baseFare                                0
totalFare                               0
seatsRemaining                          0
totalTravelDistance                     0
segmentsDepartureTimeEpochSeconds       0
segmentsDepartureTimeRaw                0
segmentsArrivalTimeEpochSeconds         0
segmentsArrivalTimeRaw                  0
segmentsArrivalAirportCode              0
segmentsDepartureAirportCode            0
segmentsAirlineName                     0
segmentsAirlineCode                     0
segmentsEquipmentDescription      

### Function to change True and False to 1 and 0

In [125]:
def map_true_false(df, columnName):
    """
        Inputs:
            df [Dataframe]: Dataframe on which the operation is performed
            columnName [String]: The column which is being modified
        
        Output:
            New Dataframe with the modified
    """
    df[columnName]=df[columnName].map(dict({True: 1, False: 0}))
    return df

data=map_true_false(data,'isBasicEconomy')
data=map_true_false(data,'isRefundable')
data=map_true_false(data,'isNonStop')
data

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,e95cef0009893d65558d17324e468aea,2022-06-01,2022-06-15,PHL,JFK,SUAJZNB3,PT5H17M,0,1,0,...,1655309340||1655323200,2022-06-15T12:09:00.000-04:00||2022-06-15T16:00:00.000-04:00,BOS||JFK,PHL||BOS,American Airlines||American Airlines,AA||AA,Airbus A321||AIRBUS INDUSTRIE A321 SHARKLETS,5160||4920,280||185,coach||coach
1,778d47d0785023302cd075d735d27db8,2022-08-11,2022-08-20,CLT,ATL,TA7NA0MC,PT14H,1,0,0,...,1661037300||1661050560||1661081220,2022-08-20T19:15:00.000-04:00||2022-08-20T22:56:00.000-04:00||2022-08-21T07:27:00.000-04:00,DTW||IND||ATL,CLT||DTW||IND,Delta||Delta||Delta,DL||DL||DL,Boeing 717||Airbus A321||Boeing 737-900,6480||4020||5220,505||241||434,coach||coach||coach
2,55ca7cc1d822f310963a25d84656bb47,2022-06-26,2022-07-01,OAK,JFK,HA0NA0MC,PT8H13M,0,0,0,...,1656706200||1656730740,2022-07-01T13:10:00.000-07:00||2022-07-01T22:59:00.000-04:00,LAX||JFK,OAK||LAX,Delta||Delta,DL||DL,Embraer 175 (Enhanced Winglets)||,5040||20040,338||2458,coach||coach
3,598db4391eb0bc5752b09c871111a5ce,2022-05-14,2022-05-23,BOS,EWR,QAA3OFEN,PT1H41M,0,0,0,...,1653347940,2022-05-23T19:19:00.000-04:00,EWR,BOS,United,UA,Airbus A319,6060,185,coach
4,e09441b23c76e8a35be3acb61d8e3e95,2022-09-25,2022-10-13,ATL,BOS,TAUNX0BC,PT10H32M,0,1,0,...,1665667500||1665697620,2022-10-13T09:25:00.000-04:00||2022-10-13T17:47:00.000-04:00,JFK||BOS,ATL||JFK,Delta||Delta,DL||DL,Boeing 737-900||Airbus A220-100,7800||5220,762||185,coach||coach
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,cfd0ca3c7574a87f1ce1dfa83d1a053b,2022-07-02,2022-07-15,LAX,CLT,HA7OA0MQ,PT9H10M,1,0,0,...,1657969320||1657986300,2022-07-16T07:02:00.000-04:00||2022-07-16T11:45:00.000-04:00,DTW||CLT,LAX||DTW,Delta||Delta,DL||DL,Airbus A321||Boeing 717,16020||6300,1985||505,coach||coach
499996,3c671c288391b26e5888c84ef143424d,2022-05-04,2022-06-23,CLT,LAX,UAVNA0MC,PT14H11M,0,0,0,...,1655982720||1655995500||1656029760,2022-06-23T07:12:00.000-04:00||2022-06-23T09:45:00.000-05:00||2022-06-23T17:16:00.000-07:00,ATL||DFW||LAX,CLT||ATL||DFW,Delta||Delta||Delta,DL||DL||DL,Boeing 717||Airbus A321||Airbus A320,4020||8100||11760,228||725||1238,coach||coach||coach
499997,2313cf9772214eaddf14651c8847eaeb,2022-08-14,2022-08-26,BOS,OAK,QAA0OHEN,PT11H11M,0,0,0,...,1661558040||1661573100||1661583120,2022-08-26T18:54:00.000-05:00||2022-08-26T22:05:00.000-06:00||2022-08-26T23:52:00.000-07:00,IAH||SLC||OAK,BOS||IAH||SLC,United||Delta||Delta,UA||DL||DL,Airbus A320||Airbus A319||Airbus A220-100,15180||11460||6780,1602||1204||588,coach||coach||coach
499998,9141b51058bfbdf9adc91abb2be9078c,2022-08-09,2022-09-06,EWR,SFO,L7AHZNN1,PT8H13M,0,0,0,...,1662485760||1662506040,2022-09-06T12:36:00.000-05:00||2022-09-06T16:14:00.000-07:00,ORD||SFO,EWR||ORD,American Airlines||American Airlines,AA||AA,Boeing 737-800||Boeing 737-800,9300||16740,720||1847,coach||coach


### Function to remove outliers

`remove_outliers` function remove's outliers depending on the given condition

In [126]:
def remove_outliers(data, variable, lower=-np.inf, upper=np.inf):
  
    """
    Input:
      data (data frame): the table to be filtered
      variable (string): the column with numerical outliers
      lower (numeric): observations with values lower than this will be removed
      upper (numeric): observations with values higher than this will be removed
    
    Output:
      a data frame with outliers removed
      
    Note: This function should not change mutate the contents of data.
    """  
    data_2=data[data[variable]<= upper]
    data_3=data_2[data_2[variable]>= lower]
    return data_3

In [127]:
data = remove_outliers(data, 'baseFare', lower = 60)
data.loc[:,'baseFare'].sort_values()

252841      60.47
423629      60.47
282635      60.47
218454      60.47
357654      60.47
           ...   
373476    3674.42
270007    4407.44
423259    4407.44
210383    4407.44
403847    4557.21
Name: baseFare, Length: 455587, dtype: float64

### rename columns

### convert date time column to date time format

### find duration of the flight in mins

### Extract segments using regex 

In [130]:
def extractSegments(dataFrame, columnName):
    """
    Inputs:
        df [Dataframe]: Dataframe on which the operation is performed
        columnName [String]: The column which is being modified
        
    Output:
        New Dataframe with the modified
    """
    #Pattern for seperating the segments
    segmentPattern=r"([^||]+)"
    
    #print(columnName)
    #print(dataFrame[columnName].str.findall(segmentPattern).str[0], dataFrame[columnName].str.findall(segmentPattern).str[1], dataFrame[columnName].str.findall(segmentPattern).str[2])
    
    #Create new columns for the extracted segments
    dataFrame[columnName+"1"]=dataFrame[columnName].str.findall(segmentPattern).str[0]
    dataFrame[columnName+"2"]=dataFrame[columnName].str.findall(segmentPattern).str[1]
    dataFrame[columnName+"3"]=dataFrame[columnName].str.findall(segmentPattern).str[2]
    return dataFrame

columnsWithSegments=[iterator for iterator in data if iterator.startswith('segments')]

for i in columnsWithSegments:
    data=extractSegments(data, i)

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Exploratory Data Analysis

### Histogram to find outliers- Chris

### find the Airport with highest traffic/best connection- Viveka

### Price of flight at different hour/duration-Chris

### min/average distance people prefer flights- Viveka

### whether the average flight is fully booked /average% of seat booked- Chirag

### top 5 most popular airlines- Chirag

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Prediction of____

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Validation of model

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Inference and Conclusion